In [ ]:
pip install opencv-python numpy matplotlib scipy

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import random
import csv

# Parameters
input_folder = 'Hands'  # Folder containing RGB images
output_folder = 'multispectral_dataset'  # Folder to save the patches

num_bands = 8  # Number of simulated spectral bands
wavelengths = np.linspace(400, 700, num_bands)  # Simulated wavelengths (400-700nm)
patch_size = (16, 16)  # Patch size to extract from each image
patches_per_image = 500  # Number of patches to extract from each image

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to simulate a multispectral image from RGB using Gaussian filters
def simulate_hyperspectral_from_rgb(rgb_image, wavelengths, num_bands):
    h, w, _ = rgb_image.shape
    hyperspectral_image = np.zeros((h, w, num_bands))
    
    for i in range(num_bands):
        sigma = (wavelengths[i] - 500) / 100  # Rough approximation for spectral bands
        # Simulating spectral bands using gaussian filters on the RGB channels
        hyperspectral_image[:, :, i] = gaussian_filter(rgb_image[:, :, 1], sigma=sigma)  # Green channel base
        hyperspectral_image[:, :, i] += 0.3 * gaussian_filter(rgb_image[:, :, 0], sigma=sigma)  # Add Red
        hyperspectral_image[:, :, i] += 0.3 * gaussian_filter(rgb_image[:, :, 2], sigma=sigma)  # Add Blue
    
    return hyperspectral_image

# Function to extract random patches from the hyperspectral image
def extract_patches(hyperspectral_image, patch_size, patches_per_image):
    h, w, num_bands = hyperspectral_image.shape
    patch_h, patch_w = patch_size
    patches = []
    
    for _ in range(patches_per_image):
        # Randomly select top-left corner of the patch
        x = random.randint(0, h - patch_h)
        y = random.randint(0, w - patch_w)
        # Extract the patch
        patch = hyperspectral_image[x:x + patch_h, y:y + patch_w, :]
        patches.append(patch)
    
    return np.array(patches)

count = dict()
with open("HandInfo.csv", "r") as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)
    for row in csv_reader:
        if not row[6][0] == "d":
            continue
        count[row[0]] = count.get(row[0], 0) + 1
        if row[0] not in ["0001024", "0001050", "0001558", "0001541", "0001033", "0000000", "0001020", "0001543", "0000528", "0000596", "0001030", "0001090", "0001019", "0001547", "0001533", "0001066", "0001540", "0001035", "0001566", "0001080"]:
            continue
        else:
            if count[row[0]] < 2:
                output_folder = f"data_final/test/{row[0]}/image{count[row[0]]}"
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)
                img_filename = row[7]
                if img_filename.endswith('.jpg') or img_filename.endswith('.png'):
                    # Load RGB image
                    img_path = os.path.join(input_folder, img_filename)
                    rgb_image = cv2.imread(img_path)
                    rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)
                    rgb_image = rgb_image / 255.0  # Normalize to [0, 1]

                    # Simulate multispectral image
                    hyperspectral_image = simulate_hyperspectral_from_rgb(rgb_image, wavelengths, num_bands)

                    # Extract random patches from the hyperspectral image
                    patches = extract_patches(hyperspectral_image, patch_size, patches_per_image)

                    # Save the patches
                    for i, patch in enumerate(patches):
                        patch_filename = f'patch_{i}.npy'
                        patch_path = os.path.join(output_folder, patch_filename)
                        np.save(patch_path, patch)  # Save each patch as a .npy file

                    print(f'Saved {len(patches)} patches from {img_filename}')
            elif count[row[0]] < 15:
                output_folder = f"data_final/train/{row[0]}/image{count[row[0]]}"
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)
                img_filename = row[7]
                if img_filename.endswith('.jpg') or img_filename.endswith('.png'):
                    # Load RGB image
                    img_path = os.path.join(input_folder, img_filename)
                    rgb_image = cv2.imread(img_path)
                    rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)
                    rgb_image = rgb_image / 255.0  # Normalize to [0, 1]

                    # Simulate multispectral image
                    hyperspectral_image = simulate_hyperspectral_from_rgb(rgb_image, wavelengths, num_bands)

                    # Extract random patches from the hyperspectral image
                    patches = extract_patches(hyperspectral_image, patch_size, patches_per_image)

                    # Save the patches
                    for i, patch in enumerate(patches):
                        patch_filename = f'patch_{i}.npy'
                        patch_path = os.path.join(output_folder, patch_filename)
                        np.save(patch_path, patch)  # Save each patch as a .npy file

                    print(f'Saved {len(patches)} patches from {img_filename}')




# # Load all images from the 'hands' folder
# for img_filename in os.listdir(input_folder):
#     if img_filename.endswith('.jpg') or img_filename.endswith('.png'):
#         # Load RGB image
#         img_path = os.path.join(input_folder, img_filename)
#         rgb_image = cv2.imread(img_path)
#         rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)
#         rgb_image = rgb_image / 255.0  # Normalize to [0, 1]

#         # Simulate multispectral image
#         hyperspectral_image = simulate_hyperspectral_from_rgb(rgb_image, wavelengths, num_bands)

#         # Extract random patches from the hyperspectral image
#         patches = extract_patches(hyperspectral_image, patch_size, patches_per_image)

#         # Save the patches
#         for i, patch in enumerate(patches):
#             patch_filename = f'{os.path.splitext(img_filename)[0]}_patch_{i}.npy'
#             patch_path = os.path.join(output_folder, patch_filename)
#             np.save(patch_path, patch)  # Save each patch as a .npy file

#         print(f'Saved {len(patches)} patches from {img_filename}')

# print("Dataset creation complete.")


Saved 500 patches from Hand_0000002.jpg
Saved 500 patches from Hand_0000003.jpg
Saved 500 patches from Hand_0000004.jpg
Saved 500 patches from Hand_0000005.jpg
Saved 500 patches from Hand_0000006.jpg
Saved 500 patches from Hand_0000007.jpg
Saved 500 patches from Hand_0000008.jpg
Saved 500 patches from Hand_0000009.jpg
Saved 500 patches from Hand_0000010.jpg
Saved 500 patches from Hand_0000011.jpg
Saved 500 patches from Hand_0000012.jpg
Saved 500 patches from Hand_0000013.jpg
Saved 500 patches from Hand_0000014.jpg
Saved 500 patches from Hand_0000015.jpg
Saved 500 patches from Hand_0002300.jpg
Saved 500 patches from Hand_0002301.jpg
Saved 500 patches from Hand_0002302.jpg
Saved 500 patches from Hand_0002303.jpg
Saved 500 patches from Hand_0002304.jpg
Saved 500 patches from Hand_0002305.jpg
Saved 500 patches from Hand_0002306.jpg
Saved 500 patches from Hand_0002307.jpg
Saved 500 patches from Hand_0002308.jpg
Saved 500 patches from Hand_0002309.jpg
Saved 500 patches from Hand_0002310.jpg
